In [1]:
import os
import numpy as np
from google import genai
from google.genai import types
from dotenv import load_dotenv
from lightrag.utils import EmbeddingFunc
from lightrag import LightRAG, QueryParam
from lightrag.kg.shared_storage import initialize_pipeline_status
from sentence_transformers import SentenceTransformer

import asyncio
import nest_asyncio
nest_asyncio.apply()

# DIR to store the index files
WORKING_DIR = "./data_indexing"

load_dotenv()
gemini_key = os.getenv("GOOGLE_API_KEY")

d:\hehe\custom_lib\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if os.path.exists(WORKING_DIR):
    import shutil

    shutil.rmtree(WORKING_DIR)

os.mkdir(WORKING_DIR)


async def llm_model_func(prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs) -> str:
    client = genai.Client(api_key=gemini_key)

    if history_messages is None:
        history_messages = []
    combined_prompt = ""
    if system_prompt:
        combined_prompt += f"{system_prompt}\n"
    for msg in history_messages:
        combined_prompt += f"{msg['role']}: {msg['content']}\n"

    combined_prompt += f"user: {prompt}"

    response = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=[combined_prompt],
        config=types.GenerateContentConfig(max_output_tokens=500, temperature=0.1),
    )

    return response.text


async def embedding_func(texts: list[str]) -> np.ndarray:
    model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
    embeddings = model.encode(texts, convert_to_numpy=True)
    return embeddings


async def initialize_rag():
    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=llm_model_func,
        embedding_func=EmbeddingFunc(
            embedding_dim=768,
            max_token_size=8192,
            func=embedding_func,
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()
    return rag

rag = asyncio.run(initialize_rag())

INFO: Process 24496 Shared-Data created for Single Process
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './data_indexing\\vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './data_indexing\\vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './data_indexing\\vdb_chunks.json'} 0 data
INFO: Process 24496 initialized updated flags for namespace: [full_docs]
INFO: Process 24496 ready to initialize storage namespace: [full_docs]
INFO: Process 24496 initialized updated flags for namespace: [text_chunks]
INFO: Process 24496 ready to initialize storage namespace: [text_chunks]
INFO: Process 24496 initialized updated flags for namespace: [entities]
INFO: Process 24496 initialized updated flags for namespace: [relationships]
INFO: Process 24496 initialized updated flags for namespace: [chunks]
INFO: Process 24496 initialized updated fla

In [ ]:
import time
import json
import logging

# Set the logging level to WARNING for specific libraries
logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
logging.getLogger("google_genai").setLevel(logging.WARNING)

with open('documents.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)  
documents = list(documents.values())
    
idx = 0
while idx < len(documents):
    doc = documents[idx]
    try:
        rag.insert(doc)
        print(f"Inserted document {idx}")
        idx += 1 
        # if idx>300:
        #     break 
    except Exception as e:
        print(f"Error inserting document {idx}: {e}")
        time.sleep(60)  # Wait for 60 seconds before

Inserted document 0
Inserted document 1
Inserted document 2
Inserted document 3
Inserted document 4
Inserted document 5
Inserted document 6
Inserted document 7
Inserted document 8
Inserted document 9
Inserted document 10
Inserted document 11
Inserted document 12
Inserted document 13
Inserted document 14
Inserted document 15
Inserted document 16
Inserted document 17
Inserted document 18
Inserted document 19
Inserted document 20
Inserted document 21
Inserted document 22
Inserted document 23
Inserted document 24
Inserted document 25
Inserted document 26
Inserted document 27
Inserted document 28
Inserted document 29
Inserted document 30
Inserted document 31
Inserted document 32
Inserted document 33
Inserted document 34
Inserted document 35
Inserted document 36
Inserted document 37
Inserted document 38
Inserted document 39
Inserted document 40
Inserted document 41
Inserted document 42
Inserted document 43
Inserted document 44
Inserted document 45
Inserted document 46
Inserted document 47
In

In [ ]:
# # thêm địa chỉ trường học vào graph
# import json
# with open('addresses.json', 'r', encoding='utf-8') as f:
#     document = json.load(f)
    
# geos = list(set(document.values()))
# mien_bac = ['Hà Nội', 'Hải Phòng', 'Bắc Ninh', 'Thái Nguyên', 'Thái Bình', 'Hải Dương', 'Vĩnh Phúc']
# mien_trung = ['Huế', 'Thừa Thiên Huế', 'Đà Nẵng', 'Thanh Hóa', 'Nghệ An', 'Khánh Hòa', 'Đà Lạt']
# mien_nam = ['Thành phố Hồ Chí Minh', 'Cần Thơ', 'Đồng Nai', 'Bình Dương', 'Long An', 'Hưng Yên']

# def locate_geo(geo):
#     if geo in mien_bac:
#         return "Miền Bắc"
#     elif geo in mien_trung:
#         return "Miền Trung"
#     elif geo in mien_nam:
#         return "Miền Nam"
#     else:
#         return None
# for locate in ["Miền Bắc", "Miền Trung", "Miền Nam"]:
#     try:
#         entity = rag.create_entity(locate, { 
#         "description": f"{locate} là một miền của Việt Nam.",
#         "entity_type": "geo",
#         })
#     except Exception as e:
#         print(f"Error creating entity for {locate}: {e}")

# for geo in geos:
#     try:
#         entity = rag.create_entity(geo,{ 
#         "description": f"{geo} là một tỉnh thuộc {locate_geo(geo)} của Việt Nam.",
#         "entity_type": "geo",
#         })
        
#         relation = rag.create_relation(geo, locate_geo(geo), {
#         "description": f"{geo} là một tỉnh thuộc {locate_geo(geo)} của Việt Nam.",
#         "keywords": f"{locate_geo(geo)}, {geo}, tỉnh, thành phố",
#         "weight": 10.0
#         })
#     except Exception as e:
#         print(f"Error creating entity for {geo}: {e}")
                
# for school, geo in document.items():
#     try:
#         relation = rag.create_relation(school, geo, {
#         "description": f"{school} là một trường học tại {geo}.",
#         "keywords": f"{geo}, {school}, tỉnh, thành phố",
#         "weight": 10.0
#         })
#     except Exception as e:
#         print(f"Error creating relation for {school} {geo}: {e}")

In [ ]:
response = rag.query(
    query= "điểm các ngành của trường đại học bách khoa",
    param=QueryParam(mode="hybrid", top_k=5, response_type="single line", only_need_context=False),
)

print(response)